In [1]:
import numpy as np
import pandas as pd
from collections import defaultdict
from difflib import SequenceMatcher
import difflib
from unidecode import  unidecode
from urllib.parse import unquote
import pickle
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import minmax_scale

In [2]:
"""
A team has its own vector for each game.
consists of table values, lineup and values of the team (part 1)
other info about the past(game statistics) (part 2)

A match consists of 2 vectors: team A and team B
In addition there is a betting odds vector: values how game will end

Save the 18 as dict or enum, quasi key= team name, value= matrix number
"""
matrix= np.zeros((18,34,72))
 

In [3]:
"""
input: team and matchday
table values in dict, key= matchday(1-34) und then look for team
Find team with similar function 
output: table position, win, loss, draw, difference as List
"""

def Tabellenwerte(nummer,mannschaft):
    #nummer = nummer -1
    if nummer == 0:
        return [0,0,0,0,0]
    else:
        tabelle_rohdaten= pd.read_pickle("Tabelle20.pkl")
        tabelle= tabelle_rohdaten.to_dict()
        spieltag= tabelle[nummer]
        teams= spieltag.keys()
        current_mannschaft=similar(mannschaft,teams)
        return spieltag[current_mannschaft]

"""
input:team(which will be find), List(all teams)
similarity search is performed
output: greatest similarity ==> team name in the form of the list
"""
def similar(mannschaft, liste):
    wert=0
    currentteam="nichts"
    for team in liste:
        similarity= SequenceMatcher(None, mannschaft, team).ratio()
        #print(team, similarity)
        if wert < similarity:
            wert= similarity
            currentteam= team
    #print(currentteam)        
    return currentteam

#Tabellenwerte(3,"münchen")

In [4]:
"""
input: team and matchday number
output: players from this matchday with their data information
"""         
def Geswerte(nummer,mannschaft):
    ges_rohdaten= pd.read_pickle("FifaGes20.pkl")
    ges= ges_rohdaten.to_dict()
    teams= (ges.keys())
    current_mannschaft= similar(mannschaft,teams)
    for key,values in ges[current_mannschaft].items():
        if key==(nummer-1):
            return values
    
#Geswerte(10,"leverkusen")



In [1]:
"""
correction of id
input: list
output: new list
"""

def Korrektur(liste):
    for i in liste:
        if str(i)=="ooooooooooooooo":
            index= liste.index(i)
            liste[index]='70'
    return liste
def Aufstellung(nummer,mannschaft):
    aufstellung_rohdaten= pd.read_pickle("GES20.pkl")

    aufstellung=(aufstellung_rohdaten.to_dict()[nummer])
    
    #print(aufstellung)
    teams= aufstellung.keys()

    current_mannschaft= similar(mannschaft,teams)
    for key,values in aufstellung.items():
        if key==current_mannschaft:
            values= Korrektur(values)
            return values




In [2]:
"""
correction of id
input: list
output: new list
"""
def Korrektur(liste):
    for i in liste:
        if str(i)=="augsburg":
            index= liste.index(i)
            liste[index]='66'
    return liste

"""
parameter: Team(1-34) and Matchday
values in a dict[1], there are all games(612(34*9*2))
separate them in blocks of 18
go through all the numbers of the teams and then key= team
"""
def Aufstellung(nummer,mannschaft):
    aufstellung_rohdaten= pd.read_pickle("GES19.pkl")

    aufstellung=(aufstellung_rohdaten.to_dict()[nummer])
    
    #print(aufstellung)
    teams= aufstellung.keys()

    current_mannschaft= similar(mannschaft,teams)
    for key,values in aufstellung.items():
        if key==current_mannschaft:
            values= Korrektur(values)
            return values

"""
input: matchday, team
each line contains matchday number and two teams
query game day and then search team by row
take average value of the previous games, with game day 1 thus 0*13
In addition, specify whether the searched team is the home/guest team
specify the opponent team
output: list, home team, away team, result of game
"""
def Statistik(nummer,mannschaft):
    
    templiste=[]
    with (open("Statistik2020.pkl", "rb")) as openfile:
        while True:
            try:
                statistik= pickle.load(openfile)
                if statistik[0]=="Spieltag":
                    continue
                if int(statistik[0])==int(nummer):
                    templiste.append(statistik)

            except EOFError:
                break
    teams=[]
    for i in templiste:
        teams.append(i[1])
        teams.append(i[2])

    #print(templiste)
    current_mannschaft= similar(mannschaft,teams)
    #print(current_mannschaft)
    
    for i in templiste:
        if str(i[1])==current_mannschaft:
            liste =i[4]

            #Sieg, Niederlage, Unentschieden herausfinden
            homegoals= int(i[4][0])
            awaygoals= int(i[5][0])
            diff= homegoals-awaygoals
            if diff<0:
                spielstand=[0,0,1]
            elif diff==0:
                spielstand=[0,1,0]
            else:
                spielstand=[1,0,0]

            #Statistiken von Spielstand für mannschaft holen
            for j in liste:
                wert=str(j).replace("km","").strip()
                wert=str(wert).replace("%","").strip()
                wert=str(wert).replace(",",".").strip()
                liste[liste.index(j)]=wert
            #Statistiken, Mannschaftsname, Ort, Spielstand
            return liste, i[2],"heim",spielstand
        elif str(i[2])==current_mannschaft:
            liste=i[5]

            #Sieg, Niederlage, Unentschieden herausfinden
            homegoals= int(i[4][0])
            awaygoals= int(i[5][0])
            diff= homegoals-awaygoals
            if diff<0:
                spielstand=[0,0,1]
            elif diff==0:
                spielstand=[0,1,0]
            else:
                spielstand=[1,0,0]

            #Statistiken von Spielstand für mannschaft holen    
            for j in liste:
                wert=str(j).replace("km","").strip()
                wert=str(wert).replace("%","").strip()
                wert=str(wert).replace(",",".").strip()
                liste[liste.index(j)]=wert
            return liste, i[1],"gast",spielstand


#Statistik(1,"Dortmund")

In [3]:
"""
input: matchday, team
each line contains matchday number and two teams
query game day and then search team by row
take average value of the previous games, with game day 1 thus 0*13
In addition, specify whether the searched team is the home/guest team
specify the opponent team
output: list, home team, away team, result of game
"""
def Daten_Teil1(mannschaft):
    spieltage=[]
    for i in range(1,35):
        #Vektor A
        #print(i)
        tabellenwerte_a= Tabellenwerte((i-1),mannschaft)
        geswerte_a= Geswerte(i,mannschaft)
        aufstellung_a= Aufstellung(i,mannschaft) 
        teil1_a= tabellenwerte_a + geswerte_a + aufstellung_a
        
        teil2_a,gegner,ort,spielstand_a= Daten_Teil2(i,mannschaft)
        #print(i)
        #print(gegner)
        alle_werte_A= teil1_a+teil2_a
        
        #Vektor B
        tabellenwerte_b= Tabellenwerte((i-1),gegner)
        geswerte_b= Geswerte(i,gegner)
        aufstellung_b= Aufstellung(i,gegner) 
        teil1_b= tabellenwerte_b + geswerte_b + aufstellung_b
        
        teil2_b,gegner_egal,ort_egal,spielstand_b= Daten_Teil2(i,gegner)
        alle_werte_B= teil1_b+teil2_b
        
        #print(mannschaft,gegner)
        #Vektor C
        teil3= Wettquoten(i,mannschaft)
        
        #Korrektur Anfang
        if type(teil3)==list:
            nichts=1
        elif i==22 and mannschaft=="Borussia Dortmund" or i==22 and gegner=="Borussia Dortmund":
            teil3= [1.4,5.5,6.5]
        elif i==28 and mannschaft=="Borussia Dortmund" or i==28 and gegner=="Borussia Dortmund":
            teil3= [3.6,4.33,1.85]
        elif (i==26 and mannschaft=="RB Leipzig") or (i==26 and mannschaft=="SC Freiburg"):
            teil3= [1.34,5.52,8.90]
        elif (i==29 and mannschaft=="SC Freiburg") or (i==29 and mannschaft=="Bayer 04 Leverkusen"):
            teil3= [1.34,5.52,8.90]
        elif (i==23 and mannschaft=="Fortuna Düsseldorf") or (i==23 and mannschaft=="Hertha BSC"):
            teil3= [1.9,3.76,4.00]
        #Korrektur Ende
        #print(mannschaft,gegner)
        if ort=="heim":
            #print(alle_werte_A,alle_werte_B,teil3)
            alle_werte= alle_werte_A+alle_werte_B+teil3+spielstand_a+[mannschaft,gegner]
            spieltage.append(alle_werte)
        elif ort=="gast":
            alle_werte= alle_werte_B+alle_werte_A+teil3+spielstand_b+[gegner,mannschaft]
            spieltage.append(alle_werte)
        
    return spieltage

        
        

"""
input: number of match day, team
Get average of the values of the past games.
so x-1,x-2,... until 1st matchday
output: Statistic values, opponent, location, result
"""
def Daten_Teil2(spiel, mannschaft):
    statistikwerte,gegner,ort,spielstand = Statistik(spiel,mannschaft)
    return statistikwerte,gegner,ort,spielstand

"""
input: number of match day, team
Get bet quotes of the game.
output: bet quote
""" 
def Wettquoten(nummer,mannschaft):
    templiste=[]
    
    liste= (pd.read_pickle("Wettquoten2020Neu.pkl")).values.tolist()
    
    teams= []
    
    #Verschiebtes Spiel umsortieren
    for i in liste: 
        if i[0]==25 and i[1]== 'B. Monchengladbach' and i[2]== 'FC Koln':
            liste[liste.index(i)]= [21, 'B. Monchengladbach', 'FC Koln', ['1.92', '3.86', '3.84']]
        if i[0]==21 and i[1]== 'Dortmund' and i[2]== 'Eintracht Frankfurt':
            liste[liste.index(i)]= [22, 'Dortmund', 'Eintracht Frankfurt', ['1.43', '5.13', '6.91']]
        if i[0]==29 and i[1]== 'Werder Bremen' and i[2]== 'Eintracht Frankfurt':
            liste[liste.index(i)]= [24, 'Werder Bremen', 'Eintracht Frankfurt', ['2.94', '3.51', '2.43']]

        
    #Spiele neu sortieren
    start=22
    for i in liste:
        if i[0]=="Spieltag":
            continue
        if int(i[0]) > 21:
            if i[0]==start:
                index= liste.index(i)
            elif i[0]!= start and start <29:
                
                start+=1
                templiste= liste[index]
                templiste[0]= start
                liste[index]= templiste
                
    #Verschiebtes Spiel umsortieren
    for i in liste:
        if i[0]=="Spieltag":
            continue
        if i[0]==24 and i[1]== 'Paderborn' and i[2]== 'FC Koln':
            liste[liste.index(i)]= [25, 'Paderborn', 'FC Koln', ['2.94', '3.51', '2.43']]
    for mannschaften in liste[1:10]:
        teams.append(mannschaften[1])
        teams.append(mannschaften[2])
    
    current_mannschaft= similar(mannschaft,teams)

    spieltagsnummer=1
    counter=0
    for spiel in liste:
        if spiel[0]== nummer and spiel[1]==current_mannschaft or spiel[0]==nummer and spiel[2]==current_mannschaft:
            #print("HIer")
            return spiel[3]
    
#Wettquoten(24, "Leverkusen")


In [8]:
'''
input:-
fill matrix 
each match in each match day for the hole season
output: matrix, list 
'''
def matrix_fuellen():
    tabelle_rohdaten= pd.read_pickle("Tabelle20.pkl")
    tabelle= tabelle_rohdaten.to_dict()
    liste= list(tabelle[1].keys())
    for mannschaft in liste:
        index= liste.index(mannschaft)
        replace0= mannschaft.replace("(M, P)","")
        replace1= replace0.replace("(M)","")
        replace2= replace1.replace("(P)","")
        replace3= replace2.replace("(N)","")


        mannschaft_new= replace3.strip()
        liste[index]= mannschaft_new
    #print(liste)

    for mannschaft in range(0,len(matrix)):
        #Aufruf von Daten für eine Mannschaft, und diese dann einfügen
        #print(liste[mannschaft])
        daten= Daten_Teil1(liste[mannschaft])
        #print(liste[mannschaft], daten)
        #print(daten)
        for spieltag in range(0,len(matrix[0])):
            mannschaft_heim= daten[spieltag][len(daten[spieltag])-2]
            mannschaft_gast= daten[spieltag][len(daten[spieltag])-1]
            #print(liste.index(mannschaft_heim.strip()),liste.index(mannschaft_gast.strip()))
            
            daten[spieltag][len(daten[spieltag])-2]= liste.index(mannschaft_heim.strip())
            daten[spieltag][len(daten[spieltag])-1]= liste.index(mannschaft_gast.strip())
            matrix[mannschaft][spieltag]= daten[spieltag]
    
    return matrix, liste


"""
input: matrix of all matches, teams
normalize all values, get all information of each feature and calculate the normalize values
output: normalize pkl file
"""
def normalise(matrix,mannschaften):

    #Alle Werte holen und diese mit minmax normalisieren

    tabellenplatz=[] #0,32
    siege=[] #1,33
    niederlage=[] #2,34
    unentschieden=[] #3,35
    diff=[] #4,36
    ges_a=[] #5,37
    ges_m=[] #6,38
    ges_v=[] #7,39
    aufstellung=[] #8-18. 40-50
    tore=[] #19, 51
    schuesse=[] #20,52
    laufleistung=[] #21,53
    paesse_ges=[] #22,54
    paesse_erfolg=[] #23,55
    paesse_fehl=[] #24,56
    passe_quote=[] #25, 57
    ballbesitz=[] #26,58
    zweikampf=[] #27, 59
    foul=[] #28, 60
    gfoul=[] #29, 61
    abseits=[] #30, 62
    ecken=[] #31, 63
    wetten=[] #32-34, 64-66

    #Alle Werte von einem Bereich holen, um diese zu normalisieren (MinMax)
    #Werte werden in richtiger Reihenfolge eingelesen, um diese danach wieder zu füllen
    for mannschaft in matrix:
        for spieltag in mannschaft:
            #tabelle
            #print(spieltag[0],spieltag[32])
            tabellenplatz.append(spieltag[0])
            tabellenplatz.append(spieltag[32])
            #siege, niederlage, unentschieden, differenz
            siege.append(spieltag[1])
            siege.append(spieltag[33])
            niederlage.append(spieltag[2])
            niederlage.append(spieltag[34])
            unentschieden.append(spieltag[3])
            unentschieden.append(spieltag[35])
            diff.append(spieltag[4])
            diff.append(spieltag[36])
            #GES werte, Angriff, Mittelfeld, Verteidigung
            ges_a.append(spieltag[5])
            ges_a.append(spieltag[37])
            ges_m.append(spieltag[6])
            ges_m.append(spieltag[38])
            ges_v.append(spieltag[7])
            ges_v.append(spieltag[39])
            #team a
            aufstellung.append(spieltag[8])
            aufstellung.append(spieltag[9])
            aufstellung.append(spieltag[10])
            aufstellung.append(spieltag[11])
            aufstellung.append(spieltag[12])
            aufstellung.append(spieltag[13])
            aufstellung.append(spieltag[14])
            aufstellung.append(spieltag[15])
            aufstellung.append(spieltag[16])
            aufstellung.append(spieltag[17])
            aufstellung.append(spieltag[18])
            #team b
            aufstellung.append(spieltag[40])
            aufstellung.append(spieltag[41])
            aufstellung.append(spieltag[42])
            aufstellung.append(spieltag[43])
            aufstellung.append(spieltag[44])
            aufstellung.append(spieltag[45])
            aufstellung.append(spieltag[46])
            aufstellung.append(spieltag[47])
            aufstellung.append(spieltag[48])
            aufstellung.append(spieltag[49])
            aufstellung.append(spieltag[50])     
            #statistiken
            tore.append(spieltag[19])
            tore.append(spieltag[51])
            schuesse.append(spieltag[20])
            schuesse.append(spieltag[52])
            laufleistung.append(spieltag[21])
            laufleistung.append(spieltag[53])
            paesse_ges.append(spieltag[22])
            paesse_ges.append(spieltag[54])
            paesse_erfolg.append(spieltag[23])
            paesse_erfolg.append(spieltag[55])
            paesse_fehl.append(spieltag[24])
            paesse_fehl.append(spieltag[56])
            passe_quote.append(spieltag[25])
            passe_quote.append(spieltag[57])
            ballbesitz.append(spieltag[26])
            ballbesitz.append(spieltag[58])
            zweikampf.append(spieltag[27])
            zweikampf.append(spieltag[59])
            foul.append(spieltag[28])
            foul.append(spieltag[60])
            gfoul.append(spieltag[29])
            gfoul.append(spieltag[61])
            abseits.append(spieltag[30])
            abseits.append(spieltag[62])
            ecken.append(spieltag[31])
            ecken.append(spieltag[63])        
            #wettquoten
            wetten.append(spieltag[64])
            wetten.append(spieltag[65])
            wetten.append(spieltag[66])


    #MinMax Scale für Tabellenpläte
    #Hierfür einmal alle Plätze (1-18) umdrehen, damit am Ende der erste Platz= 1 ist
    #print(min(tabellenplatz),max(tabellenplatz))
    feature_minmax=np.array([[min(tabellenplatz),max(tabellenplatz)],
                             [min(siege),max(siege)],
                             [min(niederlage),max(niederlage)],
                             [min(unentschieden),max(unentschieden)],
                             [min(diff),max(diff)],
                             [min(ges_a),max(ges_a)],
                             [min(ges_m),max(ges_m)],
                             [min(ges_v),max(ges_v)],
                             [min(aufstellung),max(aufstellung)],
                             [min(tore),max(tore)],
                             [min(schuesse),max(schuesse)],
                             [min(laufleistung),max(laufleistung)],
                             [min(paesse_ges),max(paesse_ges)],
                             [min(paesse_erfolg),max(paesse_erfolg)],
                             [min(paesse_fehl),max(paesse_fehl)],
                             [min(passe_quote),max(passe_quote)],
                             [min(ballbesitz),max(ballbesitz)],
                             [min(zweikampf),max(zweikampf)]])
    feature_minmax2= np.array([[min(foul),max(foul)],
                             [min(gfoul),max(gfoul)],
                             [min(abseits),max(abseits)],
                             [min(ecken),max(ecken)],
                             [min(wetten),max(wetten)]])

    #print(feature_minmax,feature_minmax2)
    
    feature_minmax_gesamt= np.vstack((feature_minmax,feature_minmax2))
    #print(feature_minmax_gesamt)
    
    ende= 19
    count=0
    for i in tabellenplatz:
        platz=tabellenplatz[count]
        if(platz== 0.0):
            count+=1
            continue
        else:
            newplatz= ende-platz
            tabellenplatz[count]=newplatz
        count+=1
    
    
    tabellenplatz_array= np.array(tabellenplatz)
    tabellenplatz_new= minmax_scale(tabellenplatz_array)
    siege_new= minmax_scale(siege)
    niederlage_new = minmax_scale(niederlage)   
    unentschieden_new= minmax_scale(unentschieden)
    diff_new= minmax_scale(diff)
    ges_a_new= minmax_scale(ges_a)
    ges_m_new= minmax_scale(ges_m)
    ges_v_new= minmax_scale(ges_v)
    aufstellung_new= minmax_scale(aufstellung)
    tore_new= minmax_scale(tore)
    schuesse_new= minmax_scale(schuesse)
    laufleistung_new= minmax_scale(laufleistung)
    paesse_ges_new= minmax_scale(paesse_ges)
    paesse_erfolg_new= minmax_scale(paesse_erfolg)
    paesse_fehl_new= minmax_scale(paesse_fehl)
    passe_quote_new= minmax_scale(passe_quote)
    ballbesitz_new= minmax_scale(ballbesitz)
    zweikampf_new= minmax_scale(zweikampf)
    foul_new= minmax_scale(foul)
    gfoul_new= minmax_scale(gfoul)
    abseits_new= minmax_scale(abseits)
    ecken_neu= minmax_scale(ecken)
    wetten_new= minmax_scale(wetten)


    #MinMax-Scale-Werte in die Matrix schreiben
    count1=0
    count2=1
    count_aufstellung1=0
    count_aufstellung2=1
    count_aufstellung3=2
    count_aufstellung4=3
    count_aufstellung5=4
    count_aufstellung6=5
    count_aufstellung7=6
    count_aufstellung8=7
    count_aufstellung9=8
    count_aufstellung10=9
    count_aufstellung11=10
    count_wetten1=0
    count_wetten2=1
    count_wetten3=2

    for mannschaft in matrix:
        for spieltag in mannschaft:
            #print(spieltag[0],spieltag[32])
            spieltag[0]=tabellenplatz_new[count1]
            spieltag[32]=tabellenplatz_new[count2]
            spieltag[1]=siege_new[count1]
            spieltag[33]=siege_new[count2]
            spieltag[2]= niederlage_new[count1]
            spieltag[34]= niederlage_new[count2]
            spieltag[3]= unentschieden_new[count1]
            spieltag[35]= unentschieden_new[count2]
            spieltag[4]= diff_new[count1]
            spieltag[36]= diff_new[count2]
            spieltag[5]= ges_a_new[count1]
            spieltag[37]= ges_a_new[count2]
            spieltag[6]= ges_m_new[count1]
            spieltag[38]= ges_m_new[count2]
            spieltag[7]= ges_v_new[count1]
            spieltag[39]= ges_v_new[count2]

            spieltag[8]= aufstellung_new[count_aufstellung1]
            spieltag[9]= aufstellung_new[count_aufstellung2]
            spieltag[10]= aufstellung_new[count_aufstellung3]
            spieltag[11]= aufstellung_new[count_aufstellung4]
            spieltag[12]= aufstellung_new[count_aufstellung5]
            spieltag[13]= aufstellung_new[count_aufstellung6]
            spieltag[14]= aufstellung_new[count_aufstellung7]
            spieltag[15]= aufstellung_new[count_aufstellung8]
            spieltag[16]= aufstellung_new[count_aufstellung9]
            spieltag[17]= aufstellung_new[count_aufstellung10]
            spieltag[18]= aufstellung_new[count_aufstellung11]
            count_aufstellung1+=11
            count_aufstellung2+=11
            count_aufstellung3+=11
            count_aufstellung4+=11
            count_aufstellung5+=11
            count_aufstellung6+=11
            count_aufstellung7+=11
            count_aufstellung8+=11
            count_aufstellung9+=11
            count_aufstellung10+=11
            count_aufstellung11+=11
            spieltag[40]= aufstellung_new[count_aufstellung1]
            spieltag[41]= aufstellung_new[count_aufstellung2]
            spieltag[42]= aufstellung_new[count_aufstellung3]
            spieltag[43]= aufstellung_new[count_aufstellung4]
            spieltag[44]= aufstellung_new[count_aufstellung5]
            spieltag[45]= aufstellung_new[count_aufstellung6]
            spieltag[46]= aufstellung_new[count_aufstellung7]
            spieltag[47]= aufstellung_new[count_aufstellung8]
            spieltag[48]= aufstellung_new[count_aufstellung9]
            spieltag[49]= aufstellung_new[count_aufstellung10]
            spieltag[50]= aufstellung_new[count_aufstellung11]
            count_aufstellung1+=11
            count_aufstellung2+=11
            count_aufstellung3+=11
            count_aufstellung4+=11
            count_aufstellung5+=11
            count_aufstellung6+=11
            count_aufstellung7+=11
            count_aufstellung8+=11
            count_aufstellung9+=11
            count_aufstellung10+=11
            count_aufstellung11+=11

            spieltag[19]= tore_new[count1]
            spieltag[51]= tore_new[count2]
            spieltag[20]= schuesse_new[count1]
            spieltag[52]= schuesse_new[count2]
            spieltag[21]= laufleistung_new[count1]
            spieltag[53]= laufleistung_new[count2]
            spieltag[22]= paesse_ges_new[count1]
            spieltag[54]= paesse_ges_new[count2]
            spieltag[23]= paesse_erfolg_new[count1]
            spieltag[55]= paesse_erfolg_new[count2]
            spieltag[24]= paesse_fehl_new[count1]
            spieltag[56]= paesse_fehl_new[count2]
            spieltag[25]= passe_quote_new[count1]
            spieltag[57]= passe_quote_new[count2]
            spieltag[26]= ballbesitz_new[count1]
            spieltag[58]= ballbesitz_new[count2]
            spieltag[27]= zweikampf_new[count1]
            spieltag[59]= zweikampf_new[count2]
            spieltag[28]= foul_new[count1]
            spieltag[60]= foul_new[count2]
            spieltag[29]= gfoul_new[count1]
            spieltag[61]= gfoul_new[count2]
            spieltag[30]= abseits_new[count1]
            spieltag[62]= abseits_new[count2]
            spieltag[31]= ecken_neu[count1]
            spieltag[63]= ecken_neu[count2]

            count1+=2
            count2+=2

            spieltag[64]= wetten_new[count_wetten1]
            spieltag[65]= wetten_new[count_wetten2]
            spieltag[66]= wetten_new[count_wetten3]
            count_wetten1+=3
            count_wetten2+=3
            count_wetten3+=3
            
    gesamt=[matrix,feature_minmax_gesamt,mannschaften]
    with open('Daten2020.pkl','wb') as f: 
        pickle.dump(gesamt, f)

matrix, mannschaften = matrix_fuellen()        
normalise(matrix,mannschaften)

#pd.read_pickle("Daten2020.pkl") 

